In [ ]:
!poetry run python -m ipykernel install \
  --user \
  --name iot-anomaly \
  --display-name "Python (iot-anomaly)"

Done.
Python 3.11.7


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# os.environ["XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"
import warnings; warnings.filterwarnings("ignore")

import math, time, pickle, logging, argparse, joblib, sys
import numpy as np, pandas as pd

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, fbeta_score
from imblearn.over_sampling import RandomOverSampler

import optuna
from tqdm.auto import tqdm
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import save_model, Model
from tensorflow.keras.optimizers import Adam

E0000 00:00:1753196585.404697   12488 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753196585.410360   12488 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753196585.423030   12488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753196585.423058   12488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753196585.423060   12488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753196585.423062   12488 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
print("GPUs:", tf.config.list_physical_devices('GPU'))

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# configurations
# path = "../data/iot_telemetry_data.csv"
path = "./data/iot_telemetry_data.csv"
model_dir = "iot_models"
os.makedirs(model_dir, exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S")
logger = logging.getLogger()

In [5]:
def load_data(path: str) -> pd.DataFrame:
    """
    Load CSV data
    """
    logger.info("Loading data from %s", path)
    df = pd.read_csv(path)
    df['ts'] = pd.to_datetime(df['ts'], unit='s', utc=True)
    return df.sort_values('ts').reset_index(drop=True)

In [6]:
def engineer_flags(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add mappings and flag features
    """
    logger.info("Adding feature mappings and flags")
    # env/device maps
    env_map = {
        "00:0f:00:70:91:0a": "stable_cooler_humid",
        "1c:bf:ce:15:ec:4d": "variable_temp_humid",
        "b8:27:eb:bf:9d:51": "stable_warmer_dry"
    }
    df['env'] = df['device'].map(env_map)
    df['device'] = df['device'].map({k: f"device_{i+1}" 
                                     for i,k in enumerate(env_map)})

    # timestamp diffs & duplicates
    df['ts_diff'] = df.groupby('device')['ts'] \
                              .diff().dt.total_seconds().fillna(0)
    # df['ts_large'] = (df['ts_diff'] > 4).astype(int)
    # df['ts_duplicate']= df.duplicated(['device','ts'], keep=False).astype(int)
    df = df.drop_duplicates(['device','ts'])

    # quantile flags per device for temp & humidity
    temp_hum = ['temp','humidity']
    quantiles = [0.01, 0.99]
    qt = {}
    for dev in df['device'].unique():
        sub = df[df['device']==dev]
        lo, hi = sub[temp_hum].quantile(quantiles).values.T
        qt[dev] = dict(temp=(math.floor(lo[0]), math.floor(hi[0])),
                       humidity=(math.floor(lo[1]), math.floor(hi[1])))
    for feat in temp_hum:
        df[f"{feat}_flag"] = 0
        for dev,(low,high) in [(d,qt[d][feat]) for d in qt]:
            mask = (df['device']==dev) & ((df[feat]<low)|(df[feat]>high))
            df.loc[mask, f"{feat}_flag"] = 1

    # gas flags
    gas_thr = {'co':0.01, 'lpg':0.01, 'smoke':0.03}
    for feat,thr in gas_thr.items():
        df[f"{feat}_flag"] = (df[feat] > thr).astype(int)

    return df

In [7]:
def preprocess(df: pd.DataFrame, test_size=0.2, random_state=42):
    """
    Feature encode, model dev preprocessing & scale.
    Returns: scaler, X_train, X_test, y_train, y_test
    """
    logger.info("Preprocessing for model training")
    
    for col in ['light','motion']:
        df[col] = df[col].astype(int)

    SENSOR_COLS = ['co','lpg','smoke','temp','humidity']
    WINDOW = max(1, int(30 / df.groupby('device')['ts_diff'].median().mean()))
    for col in SENSOR_COLS:
        roll = df.groupby('device')[col] \
                 .rolling(WINDOW, min_periods=1)
        df[f"{col}_roll_mean"] = roll.mean().reset_index(level=0, drop=True)
        df[f"{col}_roll_std"]  = roll.std().fillna(0).reset_index(level=0, drop=True)

    secs = (df['ts'].dt.hour * 3600 + df['ts'].dt.minute * 60 + df['ts'].dt.second)
    df['tod_sin'] = np.sin(2 * np.pi * secs / 86400)
    df['tod_cos'] = np.cos(2 * np.pi * secs / 86400)

    base = ['co','humidity','light','motion','lpg','smoke','temp','ts_diff']
    rolling_feats = [f"{c}_{agg}" for c in SENSOR_COLS for agg in ('roll_mean','roll_std')]
    cyclic_feats = ['tod_sin','tod_cos']
    env_ohe = pd.get_dummies(df['env'], prefix='', dtype=int)
    dev_ohe = pd.get_dummies(df['device'], prefix='', dtype=int)

    X = pd.concat([df[base + rolling_feats + cyclic_feats], env_ohe, dev_ohe], axis=1)
    X_cols = X.columns.to_list()

    flags = ['temp_flag','humidity_flag','co_flag','lpg_flag','smoke_flag']
    y = df[flags].any(axis=1).astype(int)
    logger.info("class counts:\n%s", y.value_counts().to_string())

    Xtr, Xte, ytr, yte = train_test_split(X.values, y.values, test_size=test_size, shuffle=False, random_state=random_state)

    scaler = MinMaxScaler().fit(Xtr)
    Xtr = scaler.transform(Xtr)
    Xte = scaler.transform(Xte)

    return scaler, Xtr, Xte, ytr, yte, X_cols

In [8]:
def train_usl(X_train_norm, X_test, y_test, params, random_state=42):
    """
    Runs IF + (optional) AE, fuses their anomaly scores, 
    thresholds, and evaluates on y_test.
    """
    # Fit IsolationForest on normals
    iso = IsolationForest(
        n_estimators = params["if_n_estimators"],
        max_samples = params["if_max_samples"],
        contamination = params["contamination"],
        random_state = random_state
    ).fit(X_train_norm)

    iso_tr = -iso.decision_function(X_train_norm)
    iso_te = -iso.decision_function(X_test)

    # Build & train AE only if w_ae > 0
    ae_tr = np.zeros_like(iso_tr)
    ae_te = np.zeros_like(iso_te)
    ae_model = None

    if params.get("w_ae", 0) > 0:
        input_dim = X_train_norm.shape[1]
        enc_dim =  params["ae_encoding_dim"]
        lr = params["ae_lr"]

        inp = Input(shape=(input_dim,))
        x   = Dense(enc_dim*2, activation="relu")(inp)
        x   = Dense(enc_dim,   activation="relu")(x)
        x   = Dense(enc_dim*2, activation="relu")(x)
        out = Dense(input_dim,  activation="sigmoid")(x)

        ae_model = Model(inp, out)
        ae_model.compile(optimizer=Adam(lr), loss="mse")

        ae_model.fit(
            X_train_norm, X_train_norm,
            epochs = params["ae_epochs"],
            batch_size = params["ae_batch_size"],
            validation_split = 0.1,
            shuffle = True,
            verbose = 0
        )

        Xtr_pred = ae_model.predict(X_train_norm, verbose=0)
        ae_tr     = np.mean((Xtr_pred - X_train_norm)**2, axis=1)

        Xte_pred = ae_model.predict(X_test, verbose=0)
        ae_te     = np.mean((Xte_pred - X_test)**2, axis=1)

    # Stack & scale IF+AE scores into [0,1]
    mat_tr = np.vstack([iso_tr, ae_tr]).T
    mat_te = np.vstack([iso_te, ae_te]).T
    score_scaler = MinMaxScaler().fit(mat_tr)
    norm_tr = score_scaler.transform(mat_tr)
    norm_te = score_scaler.transform(mat_te)

    # Fuse & threshold
    w_if, w_ae, thr_q = params["w_if"], params["w_ae"], params["thr_q"]
    fused_tr = w_if * norm_tr[:,0] + w_ae * norm_tr[:,1]
    fused_te = w_if * norm_te[:,0] + w_ae * norm_te[:,1]
    thr = np.quantile(fused_tr, thr_q)

    # Predict & eval
    y_pred = (fused_te >= thr).astype(int)
    metrics = {
        "roc_auc": round(roc_auc_score(y_test, fused_te), 3),
        "precision": round(precision_score(y_test, y_pred, zero_division=0), 3),
        "recall": round(recall_score(y_test, y_pred, zero_division=0), 3),
        "f1": round(f1_score(y_test, y_pred, zero_division=0), 3)
    }

    return {
        "iso_model": iso,
        "ae_model": ae_model,
        "score_scaler": score_scaler,
        "threshold": thr,
        "weights": {"if": w_if, "ae": w_ae},
        "fused_scores": fused_te,
        "y_pred": y_pred,
        "metrics": metrics
    }

In [8]:
def objective(trial):
    # USL optimization
    p = baseline_params.copy()

    p["if_n_estimators"] = trial.suggest_int("if_n_estimators", 50, 500)
    p["if_max_samples"]  = trial.suggest_categorical("if_max_samples", ["auto", 0.5, 1.0])
    p["contamination"]   = trial.suggest_float("contamination", 0.005, 0.1)
    p["ae_encoding_dim"] = trial.suggest_categorical("ae_encoding_dim", [8, 16, 32])
    p["ae_lr"]           = trial.suggest_loguniform("ae_lr", 1e-4, 1e-2)
    p["ae_epochs"]       = trial.suggest_int("ae_epochs", 10, 50)
    p["ae_batch_size"]   = trial.suggest_categorical("ae_batch_size", [64, 128, 256])

    # fusion weights & threshold
    w_if = trial.suggest_float("w_if", 0.0, 1.0)
    p["w_if"] = w_if
    p["w_ae"] = 1.0 - w_if
    p["thr_q"] = trial.suggest_float("thr_q", 0.3, 0.99)

    result = train_usl(Xtr_norm, Xte, yte, p)

    # return the F1‐score
    f1 = result["metrics"]["f1"]
    trial.set_user_attr("roc_auc", result["metrics"]["roc_auc"])
    trial.set_user_attr("precision", result["metrics"]["precision"])
    trial.set_user_attr("recall", result["metrics"]["recall"])
    logger.info(f"Trial F1={f1:.3f}")
    return f1

In [ ]:
def train_ssl(X_tr, y_tr, seed_frac, seed_cutoff, self_label_thr, C, hold_out_frac, random_state, n_splits=5):
    """
    5-fold TimeSeriesSplit CV + threshold tuning + final refit (semi-supervised).
    
    Inputs:
      X_tr, y_tr         — your full training arrays
      seed_frac          — fraction of high-conf negatives to seed
      seed_cutoff        — probability cutoff for RF “negative” seeds
      self_label_thr     — threshold for SelfTrainingClassifier’s “criterion='threshold'”
      C                  — regularization for LogisticRegression
      hold_out_frac      — % of fold-train to hold out for seeding
      random_state       — RNG seed
      n_splits           — folds in TimeSeriesSplit
      
    Prints per-fold thr & metrics, returns:
      final_model, avg_metrics, avg_thr
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics    = []
    fold_thresholds = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_tr)):
        X_tr_f, y_tr_f   = X_tr[train_idx],   y_tr[train_idx]
        X_val_f, y_val_f = X_tr[val_idx],     y_tr[val_idx]

        #  split for RF seeder & fit on larger sub-train
        sss = StratifiedShuffleSplit(n_splits=1, test_size=hold_out_frac, random_state=random_state)
        tr2_idx, hold_idx = next(sss.split(X_tr_f, y_tr_f))
        seeder = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state, n_jobs=-1)
        seeder.fit(X_tr_f[tr2_idx], y_tr_f[tr2_idx])
        p_norm_hold = seeder.predict_proba(X_tr_f[hold_idx])[:, 0]
        labels = np.full_like(y_tr_f, fill_value=-1) # build partial labels
        labels[y_tr_f == 1] = 1

        # pick the top seed_frac of hold_idx negatives
        neg_idx = hold_idx[(y_tr_f[hold_idx] == 0) & (p_norm_hold >= seed_cutoff)]
        n_seed  = max(1, int(np.ceil(seed_frac * len(neg_idx))))
        rng     = np.random.default_rng(random_state)
        seed_neg = rng.choice(neg_idx, size=n_seed, replace=False)
        labels[seed_neg] = 0

        # oversample labeled points & append unlabeled for self-training
        lab_idx = np.where(labels >= 0)[0]
        X_lab, y_lab = X_tr_f[lab_idx], labels[lab_idx]
        ros = RandomOverSampler(random_state=random_state)
        X_lab_os, y_lab_os = ros.fit_resample(X_lab, y_lab)
        unlab_idx   = np.where(labels == -1)[0]
        X_aug       = np.vstack([X_lab_os, X_tr_f[unlab_idx]])
        labels_aug  = np.concatenate([y_lab_os, labels[unlab_idx]])

        # self-train 
        base_clf = LogisticRegression(C=C, solver='saga', class_weight='balanced', max_iter=1000, random_state=random_state, n_jobs=-1)
        # base_clf = model = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=random_state)
        self_train = SelfTrainingClassifier(
            base_estimator=base_clf,
            criterion='threshold',
            threshold=self_label_thr,
            max_iter=10,
            verbose=False
        )
        self_train.fit(X_aug, labels_aug)

        # threshold sweep on val set
        probs_val  = self_train.predict_proba(X_val_f)[:,1]
        ths        = np.linspace(0.10, 0.99, 90)
        f1s        = [f1_score(y_val_f, probs_val>=t) for t in ths]
        best_i     = int(np.argmax(f1s))
        best_t     = ths[best_i]

        y_pred_f = (probs_val >= best_t).astype(int)
        metrics  = {
            "roc_auc":   roc_auc_score(y_val_f, probs_val),
            "precision": precision_score(y_val_f, y_pred_f, zero_division=0),
            "recall":    recall_score(y_val_f, y_pred_f),
            "f1":        f1s[best_i]
        }

        fold_metrics.append(metrics)
        fold_thresholds.append(best_t)
        logger.info(f"Fold {fold}: thr={best_t:.3f}, f1={metrics['f1']:.3f}")

    # aggregate CV results
    avg_metrics = {
        k: np.mean([m[k] for m in fold_metrics])
        for k in fold_metrics[0]
    }
    avg_thr = float(np.mean(fold_thresholds))
    logger.info(
        "Average CV metrics: %s   Average thr: %.3f",
        avg_metrics, avg_thr
    )

    # final seeding + self-train on X_tr, y_tr
    sss = StratifiedShuffleSplit(n_splits=1, test_size=hold_out_frac, random_state=random_state)
    tr2_idx, hold_idx = next(sss.split(X_tr, y_tr))
    seeder = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state, n_jobs=-1)
    seeder.fit(X_tr[tr2_idx], y_tr[tr2_idx])
    p_norm_hold = seeder.predict_proba(X_tr[hold_idx])[:, 0]

    labels = np.full_like(y_tr, fill_value=-1)
    labels[y_tr == 1] = 1
    neg_idx = hold_idx[(y_tr[hold_idx] == 0) & (p_norm_hold >= seed_cutoff)]
    n_seed  = max(1, int(np.ceil(seed_frac * len(neg_idx))))
    rng     = np.random.default_rng(random_state)
    seed_neg = rng.choice(neg_idx, size=n_seed, replace=False)
    labels[seed_neg] = 0

    lab_idx = np.where(labels >= 0)[0]
    X_lab, y_lab = X_tr[lab_idx], labels[lab_idx]
    ros = RandomOverSampler(random_state=random_state)
    X_lab_os, y_lab_os = ros.fit_resample(X_lab, y_lab)

    unlab_idx  = np.where(labels == -1)[0]
    X_aug      = np.vstack([X_lab_os, X_tr[unlab_idx]])
    labels_aug = np.concatenate([y_lab_os, labels[unlab_idx]])

    base_clf = LogisticRegression(C=C, solver='saga', class_weight='balanced', max_iter=1000, random_state=random_state, n_jobs=1)
    # base_clf = model = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=1, random_state=random_state)
    final_model = SelfTrainingClassifier(
        base_estimator=base_clf,
        criterion='threshold',
        threshold=self_label_thr,
        max_iter=10,
        verbose=False
    ).fit(X_aug, labels_aug)

    return final_model, avg_metrics, avg_thr


# RUN
final_model, avg_metrics, avg_thr = train_ssl(
    Xtr, ytr,
    seed_frac=0.30,
    seed_cutoff=0.90,
    self_label_thr=0.95,
    C=1.0,
    hold_out_frac=0.10,
    random_state=42,
    n_splits=5
)

probs_test = final_model.predict_proba(Xte)[:,1]
y_pred     = (probs_test >= avg_thr).astype(int)

print("Test  roc_auc:",   roc_auc_score(yte, probs_test))
print("Test  precision:", precision_score(yte, y_pred, zero_division=0))
print("Test  recall   :", recall_score(yte, y_pred))
print("Test  F1       :", f1_score(yte, y_pred))

In [9]:
def train_sl(X_tr, y_tr, X_te, y_te, C, tol, penalty, random_state, n_splits=5):
    """
    TimeSeriesSplit CV + threshold tuning + final refit.

    Inputs:
      X_tr, y_tr      — full train arrays
      C               — LR regularization param
      random_state    — for sampler & LR
      n_splits        — number of TS folds

    Prints per-fold thr & metrics, returns:
      final_model, avg_metrics_dict, avg_thr
    """
    ros = RandomOverSampler(random_state=random_state)
    X_tr_os_all, y_tr_os_all = ros.fit_resample(X_tr, y_tr)
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics    = []
    fold_thresholds = []
    
    for train_idx, val_idx in tscv.split(X_tr):
        mask = np.isin(ros.sample_indices_, train_idx)
        X_tr_os, y_tr_os = X_tr_os_all[mask], y_tr_os_all[mask]
        X_val_f, y_val_f = X_tr[val_idx], y_tr[val_idx]

        model = LogisticRegression(C=C, solver="liblinear", class_weight="balanced", max_iter=100, warm_start=True,
                                   tol=tol, penalty=penalty, n_jobs=-1, random_state=random_state)
        # model = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=1, random_state=random_state)
        model.fit(X_tr_os, y_tr_os)

        # sweep threshold on fold’s val
        probs  = model.predict_proba(X_val_f)[:,1]
        thresholds = np.linspace(0.10, 0.99, 90)
        f05s = [fbeta_score(y_val_f, probs>=t, beta=0.5) for t in thresholds]
        f1s = [f1_score(y_val_f, probs>=t) for t in thresholds]
        best_i = int(np.argmax(f1s))
        best_t = thresholds[best_i]
        best_f05 = f05s[best_i]

        y_pred = (probs >= best_t).astype(int)
        metrics = {
            "roc_auc":   round(roc_auc_score(y_val_f, probs), 3),
            "precision": round(precision_score(y_val_f, y_pred, zero_division=0), 3),
            "recall":    round(recall_score(y_val_f, y_pred, zero_division=0), 3),
            "f1":        round(f1s[best_i], 3),
            "f05":       round(best_f05, 3)
        }
        fold_metrics.append(metrics)
        fold_thresholds.append(best_t)
        # logger.info(f"Fold {fold}: thr={best_t:.3f}, f1={metrics['f1']:.3f}")

    # aggregate
    avg_metrics = {
        k: round(np.mean([m[k] for m in fold_metrics]), 3)
        for k in fold_metrics[0]
    }
    avg_thr = round(float(np.mean(fold_thresholds)), 3)
    
    # final oversampled refit on full train
    ros_full  = RandomOverSampler(random_state=random_state)
    X_full_os, y_full_os = ros_full.fit_resample(X_tr, y_tr)
    logger.info("Fitting model on oversampled train set")
    final_model = LogisticRegression(C=C, solver="liblinear", class_weight="balanced", max_iter=100, warm_start=True, 
                                     tol=tol, penalty=penalty, n_jobs=-1, random_state=random_state)
    # final_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=1, random_state=random_state)
    final_model.fit(X_full_os, y_full_os)

    # evaluate on test set
    probs_test = final_model.predict_proba(X_te)[:,1]
    y_test_pred = (probs_test >= avg_thr).astype(int)
    test_metrics = {
        "roc_auc":   round(roc_auc_score(y_te, probs_test), 3),
        "precision": round(precision_score(y_te, y_test_pred, zero_division=0), 3),
        "recall":    round(recall_score(y_te, y_test_pred, zero_division=0), 3),
        "f1":        round(f1_score(y_te, y_test_pred), 3),
        "f05":      round(fbeta_score(y_te, y_test_pred, beta=0.5), 3)
    }
    # logger.info(f"CV metrics: {avg_metrics} ‖ thr: {avg_thr}")
    # logger.info(f"SL metrics: {test_metrics}")


    return {
        "model":        final_model,
        "avg_metrics":  avg_metrics,
        "avg_thr":      avg_thr,
        "test_metrics": test_metrics
    }

In [10]:
def objective(trial):
    # Supervised learning optimization
    penalty = trial.suggest_categorical("penalty", ["l1","l2"])
    C = trial.suggest_loguniform("C", 1e-3, 1e2)
    tol = trial.suggest_loguniform("tol", 1e-4, 1e-2)

    res = train_sl(
        X_tr=Xtr,  y_tr=ytr,
        X_te=Xte,  y_te=yte,
        C=C,
        tol=tol,
        penalty=penalty,
        random_state=42,
        n_splits=5
    )
    # save CV metrics & threshold
    trial.set_user_attr("cv_metrics", res["avg_metrics"])
    trial.set_user_attr("cv_thr",     res["avg_thr"])
    return res["avg_metrics"]["f1"] # optimize CV F1

# run
# study = optuna.create_study(direction="maximize", study_name="iot-anomaly2", 
#                             sampler=optuna.samplers.TPESampler(seed=seed))
# logger.info("Starting Optuna search with %d trials", optuna_trials)
# pbar = tqdm(total=optuna_trials, desc="SL-LR tuning")
# def tqdm_callback(study, trial):
#     pbar.update(1)
#     pbar.set_postfix(best_f1=f"{study.best_value:.3f}")
# study.optimize(objective, n_trials=optuna_trials, n_jobs=4, callbacks=[tqdm_callback])
# pbar.close()

# # extract best params & metrics
# best_params = study.best_params
# best_cv = study.best_trial.user_attrs["cv_metrics"]
# best_thr = study.best_trial.user_attrs["cv_thr"]
# best_f1 = study.best_value

# logger.info("params %s", best_params)
# logger.info("metrics %s", best_cv)
# logger.info("thr %s", best_thr)
# logger.info("best CV F1 %s", best_f1)

# # final evaluation on test
# logger.info("Starting final evaluation on test set")
# final_res = train_sl(
#     X_tr=Xtr,  y_tr=ytr,
#     X_te=Xte,  y_te=yte,
#     **best_params, random_state=42, n_splits=5
# )
# logger.info("Final test metrics: %s", final_res["test_metrics"])

In [10]:
def save_artifacts(usl_base: dict, sl_base: dict, sl_base_params: dict, 
                   usl_opt: dict, sl_opt: dict, sl_opt_params: dict, scaler, model_dir: str = "iot_models"):
    """
    Persist baseline and optimized USL+SL artifacts under:
      model_dir/baseline/...  
      model_dir/optimized/...
    """
    for tag, usl_res, sl_res, sl_params in [
        ("baseline", usl_base, sl_base, sl_base_params), ("optimized", usl_opt,  sl_opt,  sl_opt_params)
        ]:
        out_dir = os.path.join(model_dir, tag)
        os.makedirs(out_dir, exist_ok=True)
        joblib.dump(scaler, os.path.join(out_dir, "scaler.pkl")) # scaler
        with open(os.path.join(out_dir, "iso.pkl"), "wb") as f: # IF
            pickle.dump(usl_res["iso_model"], f)
        ae = usl_res.get("ae_model") # Autoencoder
        if ae is not None:
            save_model(ae, os.path.join(out_dir, "autoencoder.keras"))
        ensemble_meta = {"weights": usl_res["weights"], "threshold": float(usl_res["threshold"])} # ensemble metadata
        with open(os.path.join(out_dir, "ensemble.pkl"), "wb") as f:
            pickle.dump(ensemble_meta, f)

        # supervised model
        joblib.dump(sl_res["model"], os.path.join(out_dir, "sl_model.pkl"))
        sl_meta = {"threshold": sl_res["avg_thr"], "metrics": sl_res["test_metrics"], "params": sl_params} # SL metadata
        with open(os.path.join(out_dir, "sl_meta.pkl"), "wb") as f:
            pickle.dump(sl_meta, f)
        logger.info(f"{tag.capitalize()} artifacts saved to {out_dir}")

In [11]:
# unsupervised training configs
baseline_params = {"if_n_estimators": 100, "if_max_samples": "auto", "contamination": 0.01, # IF
                   "ae_encoding_dim": 16, "ae_lr": 1e-3, "ae_epochs": 30, "ae_batch_size": 128, # AE
                   "w_if": 1, "w_ae": 0, "thr_q": 0.95 # fuse & threshold
}
# unsupervised tuned params
best_usl = {'if_n_estimators': 148, 'if_max_samples': 1.0, 'contamination': 0.07513614867821206, 
     'ae_encoding_dim': 8, 'ae_lr': 0.0004484570984975867, 'ae_epochs': 43, 'ae_batch_size': 64, 
     'w_if': 0.6582947205466332, 'thr_q': 0.9691031375838758}
best_usl['w_ae'] = 1.0 - best_usl['w_if']
for k,v in baseline_params.items():
    best_usl.setdefault(k, v)

# self-supervised learning configs
seed_frac = 0.30     
seed_cutoff = 0.90   
self_label_thr = 0.95 # self train only pseudo-labels if P ≥ 0.95
C = 1.0      
hold_out_frac = 0.10     
random_state = 42
# self-supervised tuned params
best_ssl = {'seed_frac': 0.4977861415038456, 'seed_cutoff': 0.96479545707897, 
        'self_label_thr': 0.8885603176584045, 'C': 7.259464506582171, 'hold_out_frac': 0.2419865414435976}

# supervised learning configs
sl_base_params = {"C": 1.0, "tol": 1e-4, "penalty": "l2"}
# supervised tuned params
# best_sl = {'penalty': 'l1', 'C': 11.197186694112919, 'tol': 0.00021993710180958864} #hi
best_sl = {'penalty': 'l2', 'C': 5.232594029557045, 'tol': 0.002263077789430857} #lo

optuna_trials = 100
seed = 42

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--run_optuna_usl", action="store_true")
    parser.add_argument("--run_optuna_sl", action="store_true")
    args, _ = parser.parse_known_args()

    start = time.time()
    # load & preprocess
    df = load_data(path)
    df = engineer_flags(df)
    scaler, Xtr, Xte, ytr, yte, X_cols = preprocess(df)
    Xtr_norm = Xtr[ytr == 0]  # normals only for USL

    # USL
    logger.info("== Baseline USL ==")
    usl_base = train_usl(Xtr_norm, Xte, yte, baseline_params)
    logger.info("Baseline USL metrics: %s", usl_base["metrics"])
    if args.run_optuna_usl: # USL tuning
        logger.info("Starting USL tuning with %d trials", optuna_trials)
        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
        pbar = tqdm(total=optuna_trials, desc="USL tuning")
        for _ in range(optuna_trials):
            study.optimize(objective, n_trials=1)
            pbar.update(1)
            pbar.set_postfix(best_f1=f"{study.best_value:.3f}")
        pbar.close()
        usl_best = study.best_trial.params
        for k, v in baseline_params.items(): # back-fill defaults
            usl_best.setdefault(k, v)
        logger.info("Optimized USL params: %s", usl_best)
    else:
        usl_best = best_usl
        logger.info("Skipping USL tuning; using best_usl: %s", usl_best)
    # USL final
    logger.info("== Optimized USL ==")
    usl_opt = train_usl(Xtr_norm, Xte, yte, usl_best)
    logger.info("Optimized USL metrics: %s", usl_opt["metrics"])

    # SL
    logger.info("== Baseline SL ==")
    sl_base = train_sl(X_tr=Xtr, y_tr=ytr, X_te=Xte, y_te=yte,
                        **sl_base_params, random_state=seed, n_splits=5)
    logger.info("Baseline SL test metrics: %s", sl_base["test_metrics"])
    if args.run_optuna_sl: # SL tuning
        logger.info("Starting SL tuning with %d trials", optuna_trials)
        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
        pbar = tqdm(total=optuna_trials, desc="SL tuning")
        study.optimize(objective, n_trials=optuna_trials, n_jobs=4,
                       callbacks=[lambda s, t: (pbar.update(1), pbar.set_postfix(best_f1=f"{s.best_value:.4f}"))])
        pbar.close()
        sl_best = study.best_trial.params
        logger.info("Optimized SL params: %s", sl_best)
    else:
        sl_best = best_sl
        logger.info("Skipping SL tuning; using best_sl: %s", sl_best)
    # SL final
    logger.info("== SL OPTIMIZED ==")
    sl_opt = train_sl(X_tr=Xtr,y_tr=ytr, X_te=Xte, y_te=yte,
            **sl_best, random_state=seed, n_splits=5)
    logger.info("Optimized SL test metrics: %s", sl_opt["test_metrics"])

    # save artifacts
    save_artifacts(usl_base=usl_base, sl_base=sl_base, sl_base_params=sl_base_params,
                   usl_opt=usl_opt, sl_opt=sl_opt, sl_opt_params=sl_best, scaler=scaler,
                   model_dir=model_dir
                   )
    end = time.time()
    logger.info("Runtime: %.2f minutes", (end - start) / 60)

02:04:48 INFO Loading data from ./data/iot_telemetry_data.csv
02:04:49 INFO Adding feature mappings and flags
02:04:49 INFO Preprocessing for model training
02:04:50 INFO class counts:
0    388893
1     16278
02:04:51 INFO == Baseline USL ==
02:04:53 INFO Baseline USL metrics: {'roc_auc': 0.481, 'precision': 0.026, 'recall': 0.019, 'f1': 0.022}
02:04:53 INFO Skipping USL tuning; using best_usl: {'if_n_estimators': 148, 'if_max_samples': 1.0, 'contamination': 0.07513614867821206, 'ae_encoding_dim': 8, 'ae_lr': 0.0004484570984975867, 'ae_epochs': 43, 'ae_batch_size': 64, 'w_if': 0.6582947205466332, 'thr_q': 0.9691031375838758, 'w_ae': 0.3417052794533668}
02:04:53 INFO == Optimized USL ==
I0000 00:00:1752977115.809770    2562 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14401 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:00:05.0, compute capability: 8.6
I0000 00:00:1752977117.460490    2614 service.cc:152] XLA service 0x7fc13000

In [ ]:
{'roc_auc': 0.983, 'precision': 0.761, 'recall': 0.819, 'f1': 0.789, 'f05': 0.772}
{'penalty': 'l2', 'C': 5.232594029557045, 'tol': 0.002263077789430857}

In [35]:
# dev = device_1.copy()
# time_anomalies = dev[dev['ts_diff'] <= 4]
# print(time_anomalies['ts_diff'].count())
# print(f'sum:{dev.shape[0]}')

# features = ['temp', 'humidity', 'co', 'lpg', 'smoke']
# low_qt = 0.01
# high_qt = 0.99

# for device_name in df['device'].unique():
#     device_df = df[df['device'] == device_name]
#     # print(f"\n__{device_name} (sum: {device_df[feature].count()})__")
#     for feature in features:
#         low_q = device_df[feature].quantile(low_qt)
#         high_q = device_df[feature].quantile(high_qt)
#         mean_val = device_df[feature].mean()
        
#         lowQ_count = device_df[(device_df[feature] < low_q)][feature].count()
#         highQ_count = device_df[(device_df[feature] > high_q)][feature].count()
        
        # print(
        #     f"\n{feature}\n"
        #     f"  1st: {low_q:.2f} | 99th: {high_q:.2f}\n"
        #     f"  Mean: {mean_val:.3f}\n"
        #     f"  <1st_Q: {lowQ_count} | >99th_Q: {highQ_count}")

#VIZ
# dd = df.copy()
# dd['hour'] = dd['ts'].dt.hour
# dd['min'] = dd['ts'].dt.hour * 60 + dd['ts'].dt.minute

# plt.figure(figsize=(14, 6))
# ft = 'temp'

# sns.lineplot(data=dd, x='hour', y=ft, hue='device', estimator='mean', errorbar='sd')
# # sns.scatterplot(data=dd, x='humidity', y=ft, hue='device', alpha=0.5)

# plt.title(f'Avg {ft} per day')
# plt.xlabel('hour')
# plt.ylabel(f'{ft}')
# plt.legend(title=None)
# plt.show()


# score_fused = results["fused_score"]
# thr = results["threshold"]

# plt.figure(figsize=(10, 6))

# plt.hist(score_fused[y_test == 0], bins=50, alpha=0.6, color='C0', label='Normal (y=0)') # normals
# plt.hist(score_fused[y_test == 1], bins=50, alpha=0.6, color='C3', label='Anomaly (y=1)') # anomalies
# plt.axvline(thr, color='red', linestyle='--', linewidth=2, label=f"Threshold = {thr:.3f}") # threshold line

# plt.yscale('log')
# plt.title("Distribution of Fused Anomaly Scores by Class")
# plt.xlabel("Fused Anomaly Score")
# plt.ylabel("Count")
# plt.legend()
# plt.grid(alpha=0.3)
# plt.show();